In [1]:
import numpy as np
from astropy.io import fits
import pandas as pd

In [4]:
#Read CSC 2.1 into dataframe
filename='/Users/kciurleo/Documents/kciurleo/AGN/unorganized/CSC2.1p_OIR_SDSSspecmatch.csv'
data=pd.read_csv(filename)
columns = data.columns

/var/folders/6x/hz5y30457mqg1y9dl2fg7r5m00034d/T/ipykernel_34251/3238967991.py:3: DtypeWarning: Columns (57,61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv(filename)


In [5]:
#Classified by this group as AGN, I don't actually use this
AGN = data.loc[data['SUBCLASS']=='AGN']

In [6]:
#Find only sources with SDSS data
sources = data.dropna(subset=['Sep_SPEC_CSC21P'])
print(f'All source count:{len(sources)}, {len(data)-len(sources)} not observed with SDSS')

#Find only non-extended CSC sources
point_sources = sources.loc[sources['extent_flag']==False]
print(f'Point source count:{len(point_sources)}, removed {len(sources)-len(point_sources)}')

All source count:17666, 369775 not observed with SDSS
Point source count:16022, removed 1644


In [29]:
#Write SQL Query Text
query = open('/Users/kciurleo/Documents/kciurleo/AGN/unorganized/portsmouth_query.txt','w')

#First bit of SQL; ask for objID and bpt categorization
query.write('SELECT specobjID, bpt FROM emissionLinesPort WHERE EXISTS (SELECT 1 FROM (VALUES ')

#Get all SDSS plate, MJD, fiber ID for query
values =''
for index, row in point_sources.iterrows():
    values+=f"({row['PLATE']}, {row['MJD']}, {row['FIBERID']}),"

#Remove the last comma
query.write(values[0:len(values)-1])

#Last bit of SQL
query.write(') AS t(plate_val, mjd_val, fiberID_val) WHERE plate = t.plate_val AND mjd = t.mjd_val AND fiberID = t.fiberID_val)')
query.close()